In [ ]:
import sys
import pandas as pd 
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import IPython
import sklearn 


In [4]:
from sklearn.model_selection import train_test_split
X = pd.read_csv("../data/preprocessed/trainig_data.csv")
y = pd.read_csv("../data/preprocessed/target_data.csv")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)